In [ ]:
import pandas as pd
import json
import random
from hazm import Normalizer

In [ ]:
IN_BASE_NAMES_FILE = '/Users/mohammadmahdi/Desktop/names_full.csv'
IN_CRAWLED_JL_DATA_FILE = '/Users/mohammadmahdi/Desktop/data.jl'
OUT_MAIN_COMMENTS_FILE = '/Users/mohammadmahdi/Desktop/main_comments_data.txt'
OUT_SELECTED_DATA_FILE = '/Users/mohammadmahdi/Desktop/data_final.txt'
OUT_OBFUSCATED_SELECTED_DATA_FILE = '/Users/mohammadmahdi/Desktop/data_final_obf.txt'

In [ ]:
NUMBER_OF_SAMPLES = 1000

In [ ]:
normalizer = Normalizer()

def normal(string):
    string = normalizer.normalize(string)
    string = string.replace("‌", " ")
    string = string.replace("ء", "")
    string = string.replace("أ", "ا")
    string = string.replace("آ", "ا")
    string = string.replace("إ", "ا")
    string = string.replace("ؤ", "و")
    string = string.replace("ئ", "ی")
    string = string.replace("ة", "ه")
    return string   

In [ ]:
df = pd.read_csv(IN_BASE_NAMES_FILE)
predefined_names = df['نماد.1'].tolist()
predefined_names = [normal(x) for x in predefined_names]

In [ ]:
data = []
with open(IN_CRAWLED_JL_DATA_FILE) as f:
    for line in f:
        for item in json.loads(line)['items']:
            if 'score' in item and 'content' in item: # messages of type 'twit' have no content :?
                data.append(item['content'])

In [ ]:
with open(OUT_MAIN_COMMENTS_FILE, mode='w') as f:
    i = 0
    while i < len(data):
        res = ''
        for j in range(min(len(data) - i, 100000)):
            res += data[i+j] + '\n'
        i += (j + 1)
        f.write(res)

In [ ]:
choices = random.choices(data, k=NUMBER_OF_SAMPLES)
choices = [normal(x) for x in choices]

In [ ]:
OBF = '██████████'

In [ ]:
for i in range(len(choices)):
    choices[i] = choices[i].replace('#', ' #')
names = set()
for sentiment in choices:
    split = sentiment.split()
    for word in split:
        if word.startswith('#') and word != '#':
            names.add(word[1:])

In [ ]:
names.update(predefined_names)

In [ ]:
data_obf = []
sharp_count = 0
normal_count = 0
for sentiment in choices:
    split = sentiment.split()
    for i in range(len(split)):
        for name in names:
            if ('#' + name) == split[i]:
                split[i] = '#' + OBF
                sharp_count += 1
            if name == split[i]:
                split[i] = OBF
                normal_count += 1
    data_obf.append(' '.join(split))

In [ ]:
result = ''
for record in choices:
    result += record + '\n'
    
with open(OUT_SELECTED_DATA_FILE, mode='w') as f:
    f.write(result)

In [ ]:
result = ''
for record in data_obf:
    result += record + '\n'
    
with open(OUT_OBFUSCATED_SELECTED_DATA_FILE, mode='w') as f:
    f.write(result)